### finetuning BERT for sentiment analysis

In [1]:
import os
import re
import string
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

%matplotlib inline

In [2]:
file1_path = 'inputs/nyc_listings.csv'
file2_path = 'inputs/nyc_reviews.csv'
selected1_cols = ['id', 'review_scores_rating']
selected2_cols = ['listing_id', 'comments']
df1 = pd.read_csv(file1_path, usecols=selected1_cols)
df2 = pd.read_csv(file2_path, usecols=selected2_cols)
df1.rename(columns={'id': 'listing_id'}, inplace=True)
df = pd.merge(df1, df2)
df.rename(columns={'listing_id': 'id'}, inplace=True)
df.rename(columns={'review_scores_rating': 'rating'}, inplace=True)
df.rename(columns={'comments': 'review'}, inplace=True)
new_cols = ['id', 'review', 'rating']
dfs = df[new_cols]
dfs['rating'] = dfs['rating'].round().astype(int)


positive = [4, 5]
negative = [1, 2, 3]

def sentiment(rating):
  if rating in positive:
    return 1
  else:
    return 0
  
dfs['target'] = dfs['rating'].apply(sentiment)
dfs.rename(columns={'target': 'label'}, inplace=True)



In [3]:
dfs.drop(['rating'], inplace=True, axis=1)

In [4]:
dfs.head()

,id,review,label
0,21935608,Mike was very responsive and accommodating. Th...,1
1,21935608,I couldn’t be more satisfied with my stay at M...,1
2,21935608,Mike's space was absolutely lovely and in a wo...,1
3,21935608,The apartment was just exactly what I was look...,1
4,21935608,Mike and Zoe were great hosts. Mike wrote a ve...,1


In [5]:
balance_counts = dfs.groupby('label')['label'].agg('count').values
dfs['label'].value_counts()

label
1    999846
0      1449
Name: count, dtype: int64

In [6]:
dfs.review = dfs.review.astype(str)

### building a test dataset

In [7]:
# negative sentiment
neg_dfs = dfs[(dfs["label"] == 0)]

# drop the label
neg_dfs.drop(['label'], inplace=True, axis=1)

new_dfs = dfs[(dfs["label"] == 1)].sample(300)

/tmp/ipykernel_11396/3053512889.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_dfs.drop(['label'], inplace=True, axis=1)


In [8]:
neg_dfs.value_counts()

id                   review                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [9]:
new_dfs.drop(['label'], inplace=True, axis=1)

In [10]:
new_dfs.sample(10)

,id,review
925703,13766130,"Andrew’s place is in a great location, and is ..."
372721,10165181,This is the best Air B&B we have stayed in to ...
563045,30174397,Great place! Room was super spacious! Make sur...
356375,17759694,Lindsay is the best host! She gives perfect di...
714722,52534815,Do obiektu przybyliśmy na 12 nocy. Opinie się ...
141861,6245792,.
601316,12403252,Kevin was a great host and the location was cl...
837287,675793,Amazing location! Room is on smaller side - mo...
761211,665409290103351494,העיצוב קסום והחדר חמוד. היה כיף!
30983,16055176,Everything was wonderful and I would definitel...


In [11]:
neg_dfs.head()

,id,review
321,19531174,"Most horrible Airbnb experience I ever had, we..."
363,22306777,This location is ideal for fans of Brooklyn be...
364,22306777,"Bartolomeo is a nice and friendly host, the ap..."
454,28224871,I needed a place near NY Presbyterian Hospital...
455,28224871,Unfortunately the host did not provide keys to...


In [12]:
new_dfs.head()

,id,review
79013,37885314,The place was as advertised. Masha's apartment...
24060,49946056,We had a very nice stay. This airbnb lies ver...
189301,8055721,Brady is a great host
595979,986727,The apartment was great! There were four of us...
882051,2626457,Ronnie and her son are sweethearts and very fu...


In [13]:
new_dfs = new_dfs[['id', 'review']]
neg_dfs = neg_dfs[['id', 'review']]


In [14]:
test_dfs = pd.merge(new_dfs, neg_dfs, how='outer')

In [15]:
test_dfs.sample(5)

,id,review
1381,39556412,I'll start with the positives. The apartment i...
247,11422056,Staying at Joseph's apartment was an unbelieva...
973,9944013,"La chambre est propre et agréable, bien que le..."
962,54305139,Je suis arrivé dans cet appartement pour 4 moi...
1071,49458325,I totally loved my stayed Mohammed was so kind...


In [16]:
test_dfs.review = test_dfs.review.astype(str)

In [17]:
test_dfs.sample(5)

,id,review
408,4775278,"Bonjour, \n<br/>Nous avons été accueillies par..."
74,6269945,This is an awesome place to stay in NYC. Very ...
312,13798188,Very Bad experiance! No recommendation
1540,624515382459873939,Perfect location to Manhattan College. I staye...
1295,629438804213789659,I stayed at several Airbnb's managed by Hiroki...


In [18]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [19]:
data = dfs.sample(10000)
test_data = test_dfs

In [ ]:
from sklearn.metrics import accuracy_score, roc_curve, auc

def evaluate_roc(probs, y_true):
    """
    - Print AUC and accuracy on the test set
    - Plot ROC
    @params    probs (np.array): an array of predicted probabilities with shape (len(y_true), 2)
    @params    y_true (np.array): an array of the true values with shape (len(y_true),)
    """
    preds = probs[:, 1]
    fpr, tpr, threshold = roc_curve(y_true, preds)
    roc_auc = auc(fpr, tpr)
    print(f'AUC: {roc_auc:.4f}')
       
    # Get accuracy over the test set
    y_pred = np.where(preds >= 0.5, 1, 0)
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Accuracy: {accuracy*100:.2f}%')
    
    # Plot ROC AUC
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [20]:
from sklearn.model_selection import train_test_split

X = data.review.values
y = data.label.values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=2020)

In [21]:
def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [22]:
# Print sentence 0
print('Original: ', X[0])
print('Processed: ', text_preprocessing(X[0]))

Original:  Unser Aufenthalt im " Nest" war ein Traum.Stuart und Joel haben uns bereits vor der Ankunft mit allen wichtigen Informationen versorgt, so das wir bestens informiert unseren Urlaub starten konnten. Das Appartment ist sehr liebevoll gestaltet, und der Ausblick auf den Central Partk ist traumhaft.Auch während unseres Aufenthaltes hatten wir super lieben Kontakt zu Stuart und Joel, und wenn es doch eine Frage gab, kam die Antwort innerhalb von wenigen Minuten.Sie haben uns auch tolle Tipps bezüglich der Gepäckaufbewahrung gegeben ,  nach dem Check Out (unser Flug ging nämlich erst spät abends).Auch war es uns möglich vor dem eigentlichen Check In bereits unsere Koffer im Appartment zu deponieren.Ich kann aus tiefstem Herzen das "Nest" weiterempfehlen , und meinen nächsten Urlaub in New York werde ich definitiv wieder bei Stuart und Joel verbringen wollen. Danke Euch beiden für die tolle Zeit.
Processed:  Unser Aufenthalt im " Nest" war ein Traum.Stuart und Joel haben uns bereit

In [23]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [24]:
# Concatenate train data and test data
all_data = np.concatenate([data.review.values, test_data.review.values])

# Encode our concatenated data
encoded_data = [tokenizer.encode(sent, add_special_tokens=True) for sent in all_data]

# Find the maximum length
max_len = max([len(sent) for sent in encoded_data])
print('Max length: ', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (654 > 512). Running this sequence through the model will result in indexing errors


Max length:  1124


In [25]:
# Specify `MAX_LEN`
MAX_LEN = 64

# Print sentence 0 and its encoded token ids
token_ids = list(preprocessing_for_bert([X[0]])[0].squeeze().numpy())
print('Original: ', X[0])
print('Token IDs: ', token_ids)

# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Original:  Unser Aufenthalt im " Nest" war ein Traum.Stuart und Joel haben uns bereits vor der Ankunft mit allen wichtigen Informationen versorgt, so das wir bestens informiert unseren Urlaub starten konnten. Das Appartment ist sehr liebevoll gestaltet, und der Ausblick auf den Central Partk ist traumhaft.Auch während unseres Aufenthaltes hatten wir super lieben Kontakt zu Stuart und Joel, und wenn es doch eine Frage gab, kam die Antwort innerhalb von wenigen Minuten.Sie haben uns auch tolle Tipps bezüglich der Gepäckaufbewahrung gegeben ,  nach dem Check Out (unser Flug ging nämlich erst spät abends).Auch war es uns möglich vor dem eigentlichen Check In bereits unsere Koffer im Appartment zu deponieren.Ich kann aus tiefstem Herzen das "Nest" weiterempfehlen , und meinen nächsten Urlaub in New York werde ich definitiv wieder bei Stuart und Joel verbringen wollen. Danke Euch beiden für die tolle Zeit.
Token IDs:  [101, 4895, 8043, 21200, 4765, 8865, 2102, 10047, 1000, 9089, 1000, 2162, 

/home/enton/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [26]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [32]:
%%time
import torch
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 2

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

CPU times: user 32 µs, sys: 1 µs, total: 33 µs
Wall time: 36.5 µs


In [33]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [34]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [35]:
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)

/home/enton/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.108024   |     -      |     -     |  206.05  
   1    |   40    |   0.021501   |     -      |     -     |  206.86  
   1    |   60    |   0.002950   |     -      |     -     |  203.67  
   1    |   80    |   0.021446   |     -      |     -     |  190.22  
   1    |   100   |   0.002501   |     -      |     -     |  199.00  
   1    |   120   |   0.001904   |     -      |     -     |  184.85  
   1    |   140   |   0.032010   |     -      |     -     |  189.99  
   1    |   160   |   0.021767   |     -      |     -     |  194.64  
   1    |   180   |   0.002583   |     -      |     -     |  191.44  
   1    |   200   |   0.001762   |     -      |     -     |  200.31  
   1    |   220   |   0.022005   |     -      |     -     |  210.53  


KeyboardInterrupt: 

In [ ]:
# Compute predicted probabilities on the test set
# Please initialize function `bert_predict` by running the first cell in Section 4.2.
probs = bert_predict(bert_classifier, val_dataloader)

# Evaluate the Bert classifier
evaluate_roc(probs, y_val)

In [ ]:
# Concatenate the train set and the validation set
full_train_data = torch.utils.data.ConcatDataset([train_data, val_data])
full_train_sampler = RandomSampler(full_train_data)
full_train_dataloader = DataLoader(full_train_data, sampler=full_train_sampler, batch_size=32)

# Train the Bert Classifier on the entire training data
set_seed(42)
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, full_train_dataloader, epochs=2)

In [ ]:
test_data.sample(5)

In [ ]:
# Run `preprocessing_for_bert` on the test set
print('Tokenizing data...')
test_inputs, test_masks = preprocessing_for_bert(test_data.tweet)

# Create the DataLoader for our test set
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

In [ ]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [ ]:
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, test_dataloader)

# Get predictions from the probabilities
threshold = 0.992
preds = np.where(probs[:, 1] > threshold, 1, 0)

# Number of reviews predicted non-negative
print("Number of reviews predicted non-negative: ", preds.sum())

In [ ]:
output = test_data[preds==1]
list(output.sample(20).tweet)